In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_pp_bw_linked = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Source to Use Scenarios/Brackish to Power/df_pp_bw_linked.csv')

In [ ]:
df_pp_bw_linked.columns

In [ ]:
pp_group = df_pp_bw_linked.groupby('PlantCode')
site_group = df_pp_bw_linked.groupby('site')

One powerplant to many brackish water sites. 

In [ ]:
empty = [None]*(len(list(pp_group.groups.keys())))
# Arbitrary cost $ / km /m3 ---> to be updated
transport_cost = 0.1147

one_pp_many_bw  = pd.DataFrame( {"PlantCode": list(pp_group.groups.keys()),
                                "PPLatitude": empty,
                                "PPLongitude": empty,
                                "BWsites": empty,
                                "BWavailable": empty,
                                "PPDemandFracMet" : empty,
                                "LCOW":empty, 
                                "Distances_km": empty,
                                "TransportOnM":empty,
                                "NewLCOW": empty})

one_pp_many_bw.set_index("PlantCode",inplace=True)

# Some plant demands are 0 because the capacity is 0
check_demand = lambda x: 1 if x == 0 else x

for c_name in ['PPLatitude','PPLongitude']:
    one_pp_many_bw[c_name]=empty
    for key in pp_group.groups.keys():
        one_pp_many_bw.loc[key,c_name] = pp_group.get_group(key)[c_name].values[0]
        
for key in pp_group.groups.keys():
    sites = []
    indices = []
    sub_group = pp_group.get_group(key)
    for site in sub_group['site']:
        if site not in sites:
            index = sub_group[sub_group['site']==site].index[0]
            # print(index)
            sites.append(site)
            indices.append(index)
    
    one_pp_many_bw.loc[key,"BWsites"] =  {','.join(sub_group.loc[indices]['site'])}
    # one_pp_many_bw.loc[key,"Distances_km"] = list(sub_group.loc[indices]['dist_km'])
    one_pp_many_bw.loc[key,"BWavailable"] = sum(sub_group.loc[indices]['recovery']*sub_group.loc[indices]['well_yield']/100)

    one_pp_many_bw.loc[key,"PPDemandFracMet"] = one_pp_many_bw.loc[key,"BWavailable"]/check_demand(np.average(sub_group['withm3s']))
    one_pp_many_bw.loc[key,"LCOW"] = sum(sub_group['lcow'])
    
    bw_vol_annual = sub_group.loc[indices]['recovery']*sub_group.loc[indices]['well_yield']/100*365*24*60*60   # m3/year for each site
    one_pp_many_bw.loc[key,"TransportOnM"] = sum(transport_cost*sub_group.loc[indices]['dist_km']*bw_vol_annual)  # $ / year

    one_pp_many_bw.loc[key,"NewLCOW"] = one_pp_many_bw.loc[key,"LCOW"] + one_pp_many_bw.loc[key,"TransportOnM"]/sum(bw_vol_annual)
    
one_pp_many_bw.loc[127]
# print(bw_vol_annual)

One brackish water site to many powerplants

In [ ]:
empty = [None]*(len(list(site_group.groups.keys())))
one_bw_many_pp  = pd.DataFrame( {"BWsite": list(site_group.groups.keys()),
                                "BWLatitude": empty,
                                "BWLongitude": empty,
                                "PlantCode": empty,
                                "BWAvailable": empty,
                                "PPDemand" : empty,
                                "BWutilized": empty,
                                "LCOW":empty, 
                                "Distances_km": empty,
                                "TransportOnM":empty,
                                "NewLCOW": empty})

one_bw_many_pp.set_index("BWsite",inplace=True)
# Multiple listing of same plant code but different capacity

for key in site_group.groups.keys():
    pp_list = []
    indices = []
    sub_group = site_group.get_group(key)
    for pp in sub_group['PlantCode']:
        if pp not in pp_list:
            index = sub_group[sub_group['PlantCode']==pp].index[0]
            # print(index)
            pp_list.append(pp)
            indices.append(index)
    one_bw_many_pp.loc[key]["PlantCode"] =  pp_list
    one_bw_many_pp.loc[key]["BWAvailable"] = sub_group['recovery'].values[0]*sub_group['well_yield'].values[0]/100
    one_bw_many_pp.loc[key]["PPDemand"] = sum(sub_group['withm3s'])
    if one_bw_many_pp.loc[key]["BWAvailable"]- one_bw_many_pp.loc[key]["PPDemand"] <=0:
        one_bw_many_pp.loc[key]["BWutilized"] = 1
    else:
        one_bw_many_pp.loc[key]["BWutilized"] = (one_bw_many_pp.loc[key]["BWAvailable"]- one_bw_many_pp.loc[key]["PPDemand"])/one_bw_many_pp.loc[key]["BWAvailable"]
    one_bw_many_pp.loc[key]["LCOW"] = sum(sub_group['lcow'])
    one_bw_many_pp.loc[key,"Distances_km"] = list(sub_group.loc[indices]['dist_km'])
    one_bw_many_pp.loc[key]["TransportOnM"] = sum(transport_cost*sub_group.loc[indices]['dist_km']*one_bw_many_pp.loc[key]["BWAvailable"]*365*24*3600)  # $/yr

one_bw_many_pp["NewLCOW"] = one_bw_many_pp["LCOW"] + one_bw_many_pp["TransportOnM"]/(one_bw_many_pp.loc[key]["BWAvailable"]*365*24*3600)

for c_name in ['BWLatitude','BWLongitude']:
    one_bw_many_pp[c_name]=empty
for key in site_group.groups.keys():
    one_bw_many_pp.loc[key,"BWLatitude"] = sub_group["Latitude"].values[0]
    one_bw_many_pp.loc[key,"BWLongitude"] = sub_group["Longitude"].values[0]

one_bw_many_pp.head()

In [ ]:
pp_group.get_group(108)['recovery']*pp_group.get_group(108)['well_yield']/100
test_BWavailable=pp_group.get_group(108)['recovery']*pp_group.get_group(108)['well_yield']/100
print(test_BWavailable)
test_BWavailable = sum(test_BWavailable)
print(test_BWavailable)

In [ ]:
test_metdemand_perc = pp_group.get_group(108)['withm3s']
print(test_metdemand_perc)
test_metdemand_perc = test_BWavailable / sum(pp_group.get_group(108)['withm3s'])
print(test_metdemand_perc )

In [ ]:
# Additional onm for transport

# Arbitrary cost $ / km /m3
transport_cost = 0.1
bw_vol_annual = pp_group.get_group(108)['well_yield']*(pp_group.get_group(108)['recovery'])/100*365*24*60*60 #m3/year
transport_onm_annual = transport_cost*pp_group.get_group(108)['dist_km']*bw_vol_annual # $/year

print(transport_onm_annual)

In [ ]:
for key in pp_group.groups.keys():
    print(pp_group.get_group(key)['site'])
